In [1]:
import pandas as pd
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
#Establish db connection
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [3]:
# Define the Mongo database
# Declare the db
db = client.mars_db

# Declare the collection
collection = db.mars

In [4]:
#Define executable path and initialize the browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\khagl\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


## NASA Mars News Scraping

[Mars News Site](https://redplanetscience.com/)

In [5]:
# Initialize the browser
url = 'https://redplanetscience.com/'
browser.visit(url)

In [6]:
#Parse into Beautiful Soup object
news_html = browser.html
soup = BeautifulSoup(news_html, "html.parser")

In [7]:
# Iterate through all cards of the news in the bootstrap code
# Retrieve all elements that contain News Titles and Paragraph Text
articles = soup.find_all("div", class_ = "list_text")
   
# Iterate through each card and retrieve headline and paragraph
for article in articles:
       
    # Use BeautifulSoup's find() method to navigate and retrieve attributes
    news_title = article.find("div", class_ = "content_title").text
    news_body = article.find("div", class_ = "article_teaser_body").text
    
    print("------------------------------------------")
    print(f"Headline:  {news_title}")
    print(f"Content:  {news_body}")
   
    # Dictionary to be inserted as a MongoDB document
    post = {"Headline": news_title, 
            "Content": news_body,
           }
           
    collection.insert_one(post)

------------------------------------------
Headline:  NASA's MAVEN Explores Mars to Understand Radio Interference at Earth
Content:  NASA’s MAVEN spacecraft has discovered “layers” and “rifts” in the electrically charged part of the upper atmosphere of Mars.
------------------------------------------
Headline:  NASA's Mars Perseverance Rover Gets Its Sample Handling System
Content:  The system will be collecting and storing Martian rock and soil. Its installation marks another milestone in the march toward the July launch period.
------------------------------------------
Headline:  NASA's Treasure Map for Water Ice on Mars
Content:  A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.
------------------------------------------
Headline:  NASA Invites Public to Share Excitement of Mars 2020 Perseverance Rover Launch
Content:  There are lots of ways to participate in the historic event, which is targeted for July 30.
--------------

In [8]:
# Verify database results 
results = collection.find()
for result in results:
    print(result)

{'_id': ObjectId('60625210e3fb7546755f5d5c'), 'Headline': "InSight's 'Mole' Team Peers into the Pit", 'Content': 'Efforts to save the heat probe continue.'}
{'_id': ObjectId('60625210e3fb7546755f5d5d'), 'Headline': "NASA's Mars Helicopter Attached to Mars 2020 Rover ", 'Content': 'The helicopter will be first aircraft to perform flight tests on another planet.'}
{'_id': ObjectId('60625210e3fb7546755f5d5e'), 'Headline': "Follow NASA's Perseverance Rover in Real Time on Its Way to Mars", 'Content': "A crisply rendered web application can show you where the agency's Mars 2020 mission is right now as it makes its way to the Red Planet for a Feb. 18, 2021, landing."}
{'_id': ObjectId('60625210e3fb7546755f5d5f'), 'Headline': "Mars InSight Lander to Push on Top of the 'Mole'", 'Content': 'Engineers have a plan for pushing down on the heat probe, which has been stuck at the Martian surface for a year.'}
{'_id': ObjectId('60625210e3fb7546755f5d60'), 'Headline': "NASA's Curiosity Keeps Rolling A

In [9]:
# browser.quit()

## JPL Mars Space Images Scraping

[Featured Space Image site](https://spaceimages-mars.com)

In [10]:
#Define executable path and initialize the browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\khagl\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [11]:
# Initialize the browser
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [12]:
#Parse into Beautiful Soup object
image_html = browser.html
soup = BeautifulSoup(image_html, "html.parser")

In [13]:
# Find the image url
image_path = soup.find("img", class_ = "headerimage")["src"]
print(image_path)

image/featured/mars1.jpg


In [14]:
featured_img_url = "https://spaceimages-mars.com/"+image_path
print(featured_img_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


## Mars Facts

[Mars Facts site](https://galaxyfacts-mars.com)

In [15]:
#Scrape the table using pandas
url = "https://galaxyfacts-mars.com/"
    
tables = pd.read_html(url)

tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [33]:
# First Table 
df1 = tables[0]
df1.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [34]:
# Drop the Earth column
df1 = df1.drop([2], axis=1)
df1.head()

,0,1
0,Mars - Earth Comparison,Mars
1,Diameter:,"6,779 km"
2,Mass:,6.39 × 10^23 kg
3,Moons:,2
4,Distance from Sun:,"227,943,824 km"


In [43]:
# Rename headers
df1 = df.rename(columns = {0:'Fact', 1:'Data'})
df1.head()

,Fact,Data
0,Mars - Earth Comparison,Mars
1,Diameter:,"6,779 km"
2,Mass:,6.39 × 10^23 kg
3,Moons:,2
4,Distance from Sun:,"227,943,824 km"


In [44]:
# Drop the first row
df1[df1.Fact != "Mars - Earth Comparison"]


,Fact,Data
1,Diameter:,"6,779 km"
2,Mass:,6.39 × 10^23 kg
3,Moons:,2
4,Distance from Sun:,"227,943,824 km"
5,Length of Year:,687 Earth days
6,Temperature:,-87 to -5 °C


In [55]:
# Second Table 
df2 = tables[1]
df2.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [56]:
# Rename headers
df2 = df2.rename(columns = {0:'Fact', 1:'Data'})
df2.head()

,Fact,Data
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [58]:
facts_df = df1.append(df2, ignore_index=True, sort=False)

facts_df

,Fact,Data
0,Mars - Earth Comparison,Mars
1,Diameter:,"6,779 km"
2,Mass:,6.39 × 10^23 kg
3,Moons:,2
4,Distance from Sun:,"227,943,824 km"
5,Length of Year:,687 Earth days
6,Temperature:,-87 to -5 °C
7,Equatorial Diameter:,"6,792 km"
8,Polar Diameter:,"6,752 km"
9,Mass:,6.39 × 10^23 kg (0.11 Earths)


In [59]:
# Parse to an html string
fact_table = facts_df.to_html()
fact_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Fact</th>\n      <th>Data</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Mars - Earth Comparison</td>\n      <td>Mars</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>Polar Diameter

In [21]:
browser.quit()